# Dacon 자동차 충돌 분석 AI 경진대회

- crash, ego, weather, timing 네 가지로 나누어 학습

- MMAction2
  - crash : Swin_base (video)
  - ego : Swin_base (video)
- MMClassification
  - weather : ConvNeXt_large
  - timing : EfficientNet_b0

Video Swin Transformer가 MMAction2 기반으로 구현되어있어 MMClassficiation 사용하면 패키지 관리하기 편하겠지! 했는데... <br>
mmcv 버전이 안 맞아서 가상환경 두 개를 사용했습니다.

터미널에서 code cell에 있는 것을 따라 치면 최종 submit csv를 재현할 수 있도록 작성했습니다.

In [ ]:
git clone https://github.com/miinngdok/Dacon-Car-Crash-Analysis-submission.git
cd Dacon-Car-Crash-Analysis-submission

### Directory Structure

- 데이터셋을 다운받아 아래와 같이 준비해줍니다.

In [ ]:
# car_crash_analysis
# |--- Video-Swin-Transformer
# |--- mmclassification
# |--- tools
# |--- data 
# |    |--- train 
# |    |--- test
# |    |--- train.csv
# |    |--- test.csv
# |    |--- sample_submission.csv

## Video Swin Transformer

https://github.com/SwinTransformer/Video-Swin-Transformer.git

### Install Libraries

In [ ]:
conda create -n vst python=3.9
conda activate vst
pip install torch==1.13.1
pip install torchvision==0.14.1
pip install mmcv-full==1.4.0
pip install timm
pip install scipy
pip install einops
pip install decord
pip install numpy==1.23.0
git clone https://github.com/NVIDIA/apex.git
cd apex
python setup.py install
cd ..

### Make Dataset

In [ ]:
cd tools
python make_data_mmaction2_crash.py
python make_data_mmaction2_ego.py
cd..

### Train

In [ ]:
cd Video-Swin-Transformer
python train.py ./configs/recognition/_KDI_/swin_base_crash_final.py --seed 8
python train.py ./configs/recognition/_KDI_/swin_base_ego_final.py --seed 8

### Inference

In [ ]:
python inference_custom.py swin_base_crash_final --epoch 3
python inference_custom.py swin_base_ego_final --epoch 20

In [ ]:
conda deactivate
cd ..

## MMClassification

https://github.com/open-mmlab/mmclassification.git

### Install Libraries

In [ ]:
conda create -n mmcls python=3.9
conda activate mmcls
pip install torch==1.13.1
pip install torchvision==0.14.1
pip install mmcv-full==1.7.0
pip install tqdm
pip install matplotlib

### Make Dataset

In [ ]:
cd tools
python make_data_mmcls_weather.py
python make_data_mmcls_timing.py
cd ..

### train

In [ ]:
cd mmclassification
python train.py ./configs/_KDI_/convnext_large_weather_final.py --seed 8
python train.py ./configs/_KDI_/efficientnet_b0_timing_final_0.py --seed 8
python train.py ./configs/_KDI_/efficientnet_b0_timing_final_1.py --seed 8

### Inference

In [ ]:
python inference_custom.py convnext_large_weather_final --epoch 4
python inference_custom.py efficientnet_b0_timing_final_0 --epoch 1
python inference_custom.py efficientnet_b0_timing_final_1 --epoch 1
python inference_custom.py efficientnet_b0_timing_final_1 --epoch 4

In [ ]:
cd ..

## Merge csv

In [ ]:
cd tools
python merge_csv.py